<a href="https://colab.research.google.com/github/JY522IC/PointCloudProject/blob/main/Pointnet_Segment_0603_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The original Pointnet work.
Based on the paper:https://openaccess.thecvf.com/content_cvpr_2017/papers/Qi_PointNet_Deep_Learning_CVPR_2017_paper.pdf
Code:https://github.com/Young98CN/pointconv_pytorch
Github: https://github.com/yanx27/Pointnet_Pointnet2_pytorch/blob/master/models/pointnet_utils.py


# Install the requred libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install the required libraries
!pip install open3d
!pip install provider

# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper functions for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_mesh(pos, face):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    # ax = fig.gca(projection='3d')
    ax.axes.xaxis.set_ticklabels([])
    ax.axes.yaxis.set_ticklabels([])
    ax.axes.zaxis.set_ticklabels([])
    ax.plot_trisurf(pos[:, 0], pos[:, 1], pos[:, 2], triangles=data.face.t(), antialiased=False)
    plt.show()


def visualize_points(pos, edge_index=None, index=None):
    fig = plt.figure(figsize=(4, 4))
    if edge_index is not None:
        for (src, dst) in edge_index.t().tolist():
             src = pos[src].tolist()
             dst = pos[dst].tolist()
             plt.plot([src[0], dst[0]], [src[1], dst[1]], linewidth=1, color='black')
    if index is None:
        plt.scatter(pos[:, 0], pos[:, 1], s=50, zorder=1000)
    else:
       mask = torch.zeros(pos.size(0), dtype=torch.bool)
       mask[index] = True
       plt.scatter(pos[~mask, 0], pos[~mask, 1], s=50, color='lightgray', zorder=1000)
       plt.scatter(pos[mask, 0], pos[mask, 1], s=50, zorder=1000)
    plt.axis('off')
    plt.show()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.4
    Uninstalling widgetsnbextension-3.6.4:
      Successfully uninstalled widgetsnbextension-3.6.4
  Attempting uninstall: werkzeug
    Found existing instal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.6 MB/s eta 0:00:00
2.0.1+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Make the custom dataset

In [ ]:
import torch
import os
import pandas as pd
from torch_geometric.data import InMemoryDataset, Data, download_url, extract_zip
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from tqdm import tqdm
import open3d as o3d
import numpy as np
import json

In [ ]:
def inCube(X, corners):
  """
  Check if a point `X` is inside of rectangular prison with `corners` (two points)
  """
  # Where is X > corners?
  greater = X > corners
  # If X is greater than both corners of any axis, it is outside
  inside = ~np.any(np.equal(*greater))

  return inside

In [ ]:
def pc_normalize(pc):
    """
    Point data normilization
    :param pc: the pc to normilized
    :return: the point data after normilization
    """
    # the average value
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
    # rescale
    pc = pc / m
    return pc

In [ ]:
class Operating_room_segment(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(Operating_room_segment, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        # List of the raw files
        return []

    @property
    def processed_file_names(self):
        # return list of files should be in processed dir, if found - skip processing
        return ['data.pt']

    # def download(self):
    #     # Download the file specified in self.url and store
    #     # it in self.raw_dir
    #     path = download_url(self.url, self.raw_dir)
    #     extract_zip(path, self.raw_dir)
    #     # The zip file is removed
    #     os.unlink(path)


    def process(self):

      # In the loop we extract the nodes' embeddings, edges connectivity for
      # and label for a graph, process the information and put it in a Data
      # object, then we add the object to a list
      data_list = []
      root_new = '/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Data0529/237115_import-pointclouds-ply_001_episodes_01 2/import-pointclouds-ply'

      # load the annotations
      annotation_root = os.path.join(root_new, 'annotation.json')
      with open(annotation_root, 'r') as fcc_file:
        annotation = json.load(fcc_file)

      # load the annotations
      frame_pointcloud_map_root = os.path.join(root_new, 'frame_pointcloud_map.json')
      with open(frame_pointcloud_map_root, 'r') as fcc_file:
        frame_pointcloud_map = json.load(fcc_file)

      # Define the direction for the point cloud saved
      pointcloud_data_root = os.path.join(root_new, 'pointcloud')

      # Define the device and dtype to create new point cloud data
      device = o3d.core.Device("CPU:0")
      dtype = o3d.core.float64

      # Define the color and class map
      Classes = ['Wall','Surgeon sitting on chair', 'Surgeon Standing',
      'Table1','Table2','Table3','Table4']
      # Classes = ['Wall','Surgeon sitting on chair', 'Surgeon Standing','Chair',
      # 'Table1','Table2','Table3','Table4']
      Color_class_map = {'Surgeon sitting on chair': [255, 0, 0], 'Surgeon Standing': [0, 255, 0], 'Chair': [0, 0, 255],
                        'Table1': [255, 0, 255], 'Table2': [0, 255, 255], 'Table3': [221, 160, 221], 'Table4': [0, 255, 127]}

      Class_title_map = {}
      for i in annotation['objects']:
          Class_title_map[i['key']] = i['classTitle']

      # Wall and floor is class 0
      # Class_label_map = {'Surgeon sitting on chair': 1,
      #           'Surgeon Standing': 2,
      #           'Chair': 3,
      #           'Table1': 4,
      #           'Table2': 5,
      #           'Table3': 6,
      #           'Table4': 7}
      Class_label_map = {'Surgeon sitting on chair': 1,
          'Surgeon Standing': 2,
          'Table1': 3,
          'Table2': 4,
          'Table3': 5,
          'Table4': 6}

      # Do the point cloud data process
      number_of_files = annotation['framesCount']

      # Define the labelweights
      labelweights = np.zeros(len(Classes))

      for i in range(number_of_files):

        # Define the file path
        file_index = annotation['frames'][i]['index']
        file_name = frame_pointcloud_map[str(file_index)]
        file_dir = os.path.join(root_new,'pointcloud', file_name)

        # Load the pcd file
        pcd = o3d.io.read_point_cloud(file_dir)

        # Control the number of points in the point cloud file
        pcd = pcd.farthest_point_down_sample(15000)

        point_pos = np.asarray(pcd.points)
        point_color = np.asarray(pcd.colors)

        # Define the ground truth colors matrix
        colors_ground_truth_matrix = np.full(np.shape(point_pos), fill_value=192)
        Y_labels = np.zeros(np.shape(point_pos)[0])

        # Labels
        for j in range(np.shape(point_pos)[0]):

            # Get points position
            X = np.array(point_pos[j])

            for k in annotation['frames'][i]['figures']:
                x_min = k['geometry']['position']['x'] - k['geometry']['dimensions']['x'] / 2
                x_max = k['geometry']['position']['x'] + k['geometry']['dimensions']['x'] / 2

                y_min = k['geometry']['position']['y'] - k['geometry']['dimensions']['y'] / 2
                y_max = k['geometry']['position']['y'] + k['geometry']['dimensions']['y'] / 2

                z_min = k['geometry']['position']['z'] - k['geometry']['dimensions']['z'] / 2
                z_max = k['geometry']['position']['z'] + k['geometry']['dimensions']['z'] / 2

                corners = [[x_min, y_min, z_min], [x_max, y_max, z_max]]

                if inCube(X, corners):
                    colors_ground_truth_matrix[j] = Color_class_map[Class_title_map[k['objectKey']]]
                    Y_labels[j] = Class_label_map[Class_title_map[k['objectKey']]]

        # Use histogram to calculate the sample weight
        tmp, _ = np.histogram(Y_labels, range(len(Classes)+1))
        print('tmp')
        print(tmp)
        labelweights += tmp

        # Normilize the data
        new_points = pc_normalize(point_pos)

        # Calculate the normal information
        device = o3d.core.Device("CPU:0")
        dtype = o3d.core.float64
        pcd_new = o3d.t.geometry.PointCloud(device)
        pcd_new.point.positions = o3d.core.Tensor(new_points, dtype, device)
        pcd_new = pcd_new.to_legacy()
        radius = 0.01  # Searching Radius
        max_nn = 30  # Number of neighboring points to estimate normal vector
        pcd_new.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))

        # Convert the position and label information into the Pytorch format
        Pos = torch.tensor(new_points,dtype=torch.float32)
        Y = torch.tensor(Y_labels,dtype=torch.long)
        Normals = torch.tensor(np.asarray(pcd_new.normals),dtype=torch.float32)
        File_name_index = torch.tensor(file_index,dtype=torch.long)
        Colors = torch.tensor(point_color,dtype=torch.float32)
        Colors_ground_truth = torch.tensor(colors_ground_truth_matrix,dtype=torch.float32)

        # Save the data
        graph = Data(pos = Pos, y = Y, normal = Normals, file_name_index = File_name_index, colors = Colors, colors_ground_truth = Colors_ground_truth)
        data_list.append(graph)

        # print(index)

      # Calculate the sample weight can save in the same index as dataset direction
      labelweights = labelweights.astype(np.float32)
      labelweights = labelweights / np.sum(labelweights)
      labelweights = np.power(np.amax(labelweights) / labelweights, 1 / 3.0)
      print(labelweights)
      weight_dirction = os.path.join(self.root,'SampleWeightArray.npy')
      np.save(weight_dirction,labelweights)

      # Apply the functions specified in pre_filter and pre_transform
      if self.pre_filter is not None:
          data_list = [data for data in data_list if self.pre_filter(data)]

      if self.pre_transform is not None:
          data_list = [self.pre_transform(data) for data in data_list]

      # Store the processed data
      data, slices = self.collate(data_list)
      torch.save((data, slices), self.processed_paths[0])

In [ ]:
import torch_geometric.transforms as T
# train_index = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/InterestAreaClassfication0521/Datasetmade/InterestArea0523/train0523.csv')
# test_index = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/InterestAreaClassfication0521/Datasetmade/InterestArea0523/test0523.csv')

# Create the train and test dataset based on the direction
# State = 1
trainDataset = Operating_room_segment(root='/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Dataset/Dataset0603/Train')
# State = 2
testDataset = Operating_room_segment(root='/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Dataset/Dataset0603/Test')

# Network

In [ ]:
def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace=True

In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F


class STN3d(nn.Module):
    def __init__(self, channel):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32))).view(1, 9).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x

class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k * k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1, self.k * self.k).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetEncoder(nn.Module):
    def __init__(self, global_feat=True, feature_transform=False, channel=3):
        super(PointNetEncoder, self).__init__()
        self.stn = STN3d(channel)
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        B, D, N = x.size()
        trans = self.stn(x)
        x = x.transpose(2, 1)
        if D > 3:
            feature = x[:, :, 3:]
            x = x[:, :, :3]
        x = torch.bmm(x, trans)
        if D > 3:
            x = torch.cat([x, feature], dim=2)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2, 1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2, 1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, N)
            return torch.cat([x, pointfeat], 1), trans, trans_feat


def feature_transform_reguliarzer(trans):
    d = trans.size()[1]
    I = torch.eye(d)[None, :, :]
    if trans.is_cuda:
        I = I.cuda()
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2, 1)) - I, dim=(1, 2)))
    return loss

Define the main network structure

In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import torch.nn.functional as F

class get_model(nn.Module):
    def __init__(self, num_class):
        super(get_model, self).__init__()
        self.k = num_class
        self.feat = PointNetEncoder(global_feat=False, feature_transform=True, channel=9)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans_feat

class get_loss(torch.nn.Module):
    def __init__(self, mat_diff_loss_scale=0.001):
        super(get_loss, self).__init__()
        self.mat_diff_loss_scale = mat_diff_loss_scale

    def forward(self, pred, target, trans_feat, weight):
        loss = F.nll_loss(pred, target, weight = weight)
        mat_diff_loss = feature_transform_reguliarzer(trans_feat)
        total_loss = loss + mat_diff_loss * self.mat_diff_loss_scale
        return total_loss

# Define the Main train and test structure

In [ ]:
def add_vote(vote_label_pool, point_idx, pred_label, weight):
    B = pred_label.shape[0]
    N = pred_label.shape[1]
    for b in range(B):
        for n in range(N):
            if weight[b, n] != 0 and not np.isinf(weight[b, n]):
                vote_label_pool[int(point_idx[b, n]), int(pred_label[b, n])] += 1
    return vote_label_pool

In [ ]:
import os
import sys
import torch
import numpy as np
import time
import datetime
import logging
import provider
import importlib
import shutil
import argparse

from pathlib import Path
from tqdm import tqdm
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

from sklearn.utils.class_weight import compute_sample_weight
from torch_geometric.loader import DataLoader
from torch.utils.data import WeightedRandomSampler

# Define the class and label maps
classes = ['Wall','Surgeon sitting on chair', 'Surgeon Standing',
      'Table1','Table2','Table3','Table4']
class2label = {cls: i for i, cls in enumerate(classes)}
seg_classes = class2label
seg_label_to_cat = {}

for i, cat in enumerate(seg_classes.keys()):
    seg_label_to_cat[i] = cat

# Define the required parameters
num_class = len(classes)
# Number of classes in the dataset
log_dir = None
# Define the log dir
npoint = 15000
# Point number
viusal = True
# visualize result [default: False]
num_votes = 5
# aggregate segmentation scores with voting [default: 5]

# Define the log
def log_string(str):
  logger.info(str)
  print(str)

'''HYPER PARAMETER'''
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
experiment_dir = '/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Result/Segment0603try/sem_seg/2023-06-03_21-31'
visual_dir = experiment_dir + '/visual/'
visual_dir = Path(visual_dir)
visual_dir.mkdir(exist_ok=True)

'''LOG'''
logger = logging.getLogger("Model")
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler('%s/eval.txt' % experiment_dir)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
log_string('PARAMETER ...')

# Load the data
testDataLoader = DataLoader(testDataset, batch_size=3, shuffle=True)
weights = np.load("/content/drive/MyDrive/Colab Notebooks/Segmentation0529/Dataset/Dataset0603/Test/SampleWeightArray.npy")
weights = torch.Tensor(weights)
log_string("The number of test data is: %d" % len(testDataset))

# Load the model
classifier = get_model(num_class)
checkpoint = torch.load(str(experiment_dir) + '/checkpoints/best_model.pth')
classifier.load_state_dict(checkpoint['model_state_dict'])
classifier = classifier.eval()


num_batches = len(testDataLoader)
total_correct = 0
total_seen = 0
loss_sum = 0
labelweights = np.zeros(num_class)
total_seen_class_tmp = [0 for _ in range(num_class)]
total_correct_class_tmp = [0 for _ in range(num_class)]
total_iou_deno_class_tmp = [0 for _ in range(num_class)]
if args.visual:
  fout = open(os.path.join(visual_dir, scene_id[batch_idx] + '_pred.obj'), 'w')
  fout_gt = open(os.path.join(visual_dir, scene_id[batch_idx] + '_gt.obj'), 'w')


for data in tqdm(testDataLoader, desc='Processing'):
  points = np.asarray(data.pos)
  color = np.asarray(data.colors)
  normal = np.asarray(data.normal)
  points = np.reshape(points,[int(np.shape(points)[0]/npoint),npoint,3])
  color = np.reshape(color,[int(np.shape(normal)[0]/npoint),npoint,3])
  normal = np.reshape(normal,[int(np.shape(normal)[0]/npoint),npoint,3])
  new_points = np.concatenate((points,color,normal),axis=2)
  points = torch.Tensor(new_points)
  points = points.transpose(2, 1)

  seg_pred, trans_feat = classifier(points)
  pred_val = seg_pred.contiguous().cpu().data.numpy()
  seg_pred = seg_pred.contiguous().view(-1, num_class)

  batch_label = np.asarray(data.y)
  batch_label = np.reshape(batch_label,[np.shape(points)[0],npoint])
  target = data.y

  pred_val = np.argmax(pred_val, 2)
  correct = np.sum((pred_val == batch_label))
  total_correct += correct
  total_seen += (np.shape(points)[0] * npoint)
  tmp, _ = np.histogram(batch_label, range(num_class + 1))
  labelweights += tmp

  for l in range(num_class):
    total_seen_class_tmp[l] += np.sum((whole_scene_label == l))
    total_correct_class_tmp[l] += np.sum((pred_label == l) & (whole_scene_label == l))
    total_iou_deno_class_tmp[l] += np.sum(((pred_label == l) | (whole_scene_label == l)))
    total_seen_class[l] += total_seen_class_tmp[l]
    total_correct_class[l] += total_correct_class_tmp[l]
    total_iou_deno_class[l] += total_iou_deno_class_tmp[l]

  iou_map = np.array(total_correct_class_tmp) / (np.array(total_iou_deno_class_tmp, dtype=np.float) + 1e-6)
  print(iou_map)
  arr = np.array(total_seen_class_tmp)
  tmp_iou = np.mean(iou_map[arr != 0])
  log_string('Mean IoU of %s: %.4f' % (scene_id[batch_idx], tmp_iou))
  print('----------------------------')

  filename = os.path.join(visual_dir, scene_id[batch_idx] + '.txt')
  with open(filename, 'w') as pl_save:
      for i in pred_label:
          pl_save.write(str(int(i)) + '\n')
      pl_save.close()
  for i in range(whole_scene_label.shape[0]):
      color = g_label2color[pred_label[i]]
      color_gt = g_label2color[whole_scene_label[i]]
      if args.visual:
          fout.write('v %f %f %f %d %d %d\n' % (
              whole_scene_data[i, 0], whole_scene_data[i, 1], whole_scene_data[i, 2], color[0], color[1],
              color[2]))
          fout_gt.write(
              'v %f %f %f %d %d %d\n' % (
                  whole_scene_data[i, 0], whole_scene_data[i, 1], whole_scene_data[i, 2], color_gt[0],
                  color_gt[1], color_gt[2]))
  if args.visual:
      fout.close()
      fout_gt.close()

IoU = np.array(total_correct_class) / (np.array(total_iou_deno_class, dtype=np.float) + 1e-6)
iou_per_class_str = '------- IoU --------\n'
for l in range(NUM_CLASSES):
  iou_per_class_str += 'class %s, IoU: %.3f \n' % (
      seg_label_to_cat[l] + ' ' * (14 - len(seg_label_to_cat[l])),
      total_correct_class[l] / float(total_iou_deno_class[l]))
log_string(iou_per_class_str)
log_string('eval point avg class IoU: %f' % np.mean(IoU))
log_string('eval whole scene point avg class acc: %f' % (
  np.mean(np.array(total_correct_class) / (np.array(total_seen_class, dtype=np.float) + 1e-6))))
log_string('eval whole scene point accuracy: %f' % (
      np.sum(total_correct_class) / float(np.sum(total_seen_class) + 1e-6)))

print("Done!")

<IPython.core.display.Javascript object>

INFO:Model:PARAMETER ...


PARAMETER ...


INFO:Model:The number of training data is: 3
INFO:Model:The number of test data is: 3
INFO:Model:No existing model, starting training from scratch...
INFO:Model:Start training...
INFO:Model:Epoch 1 (1/0):
INFO:Model:Learning rate:0.000100


The number of training data is: 3
The number of test data is: 3
No existing model, starting training from scratch...
Epoch 1 (1/0):
Learning rate:0.000100
BN momentum updated to: 0.100000


...

Saving model....
Best mIoU: 0.003920
Epoch 5 (5/4):
Learning rate:0.000100
BN momentum updated to: 0.100000


Processing: 100%|██████████| 1/1 [00:14<00:00, 14.23s/it]
INFO:Model:Training mean loss: 0.709024
INFO:Model:Training accuracy: 0.082756
INFO:Model:---- EPOCH 005 EVALUATION ----


Training mean loss: 0.709024
Training accuracy: 0.082756
---- EPOCH 005 EVALUATION ----


Processing: 100%|██████████| 1/1 [00:05<00:00,  5.58s/it]
INFO:Model:eval mean loss: 2.025725
INFO:Model:eval point avg class IoU: 0.020593
INFO:Model:eval point accuracy: 0.116333
INFO:Model:eval point avg class acc: 0.154550
INFO:Model:------- IoU --------
class Wall     weight: 0.054, IoU: 0.115 
class Surgeon sitting on chair weight: 0.781, IoU: 0.000 
class Surgeon Standing weight: 0.031, IoU: 0.000 
class Table1   weight: 0.041, IoU: 0.000 
class Table2   weight: 0.043, IoU: 0.000 
class Table3   weight: 0.023, IoU: 0.029 
class Table4   weight: 0.027, IoU: 0.000 

INFO:Model:Eval mean loss: 2.025725
INFO:Model:Eval accuracy: 0.116333
INFO:Model:Save model...
INFO:Model:Saving at /content/drive/MyDrive/Colab Notebooks/Segmentation0529/Result/Segment0603try/sem_seg/2023-06-03_21-31/checkpoints/best_model.pth
INFO:Model:Saving model....
INFO:Model:Best mIoU: 0.020593
INFO:Model:End of training...


eval mean loss: 2.025725
eval point avg class IoU: 0.020593
eval point accuracy: 0.116333
eval point avg class acc: 0.154550
------- IoU --------
class Wall     weight: 0.054, IoU: 0.115 
class Surgeon sitting on chair weight: 0.781, IoU: 0.000 
class Surgeon Standing weight: 0.031, IoU: 0.000 
class Table1   weight: 0.041, IoU: 0.000 
class Table2   weight: 0.043, IoU: 0.000 
class Table3   weight: 0.023, IoU: 0.029 
class Table4   weight: 0.027, IoU: 0.000 

Eval mean loss: 2.025725
Eval accuracy: 0.116333
Saving at /content/drive/MyDrive/Colab Notebooks/Segmentation0529/Result/Segment0603try/sem_seg/2023-06-03_21-31/checkpoints/best_model.pth
Saving model....
Best mIoU: 0.020593
